In [188]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.auto import tqdm
import re
import copy
import warnings
warnings.filterwarnings('ignore')

In [189]:
ls drive/MyDrive/ВКР_Рекомендательная_система/

'!03&04_17_VSE.csv'   города_для_регионов.xlsx
 выкупленные.xlsx     расставленные.xlsx


In [190]:
df_all = pd.read_excel('drive/MyDrive/ВКР_Рекомендательная_система/расставленные.xlsx')

In [191]:
def summary(x):
    result = {
        'КоличествоЧеков': x[(x.НовыйСтатус=='Доставлен')&(x.Статус=='Доставлен')&(x.Отменено=='Нет')&(x.КоличествоПроданоКлиенту>0)]['НомерЗаказаНаСайте'].nunique(),
        'СреднийЧек': x[(x.НовыйСтатус=='Доставлен')&(x.Статус=='Доставлен')&(x.Отменено=='Нет')&(x.КоличествоПроданоКлиенту>0)]['СуммаСтроки'].sum() / x[(x.НовыйСтатус=='Доставлен')&(x.Статус=='Доставлен')&(x.Отменено=='Нет')&(x.КоличествоПроданоКлиенту>0)]['НомерЗаказаНаСайте'].nunique(),
        'СреднееЧислоТоваровЧека': x[(x.НовыйСтатус=='Доставлен')&(x.Статус=='Доставлен')&(x.Отменено=='Нет')&(x.КоличествоПроданоКлиенту>0)]['Количество'].sum() / x[(x.НовыйСтатус=='Доставлен')&(x.Статус=='Доставлен')&(x.Отменено=='Нет')&(x.КоличествоПроданоКлиенту>0)]['НомерЗаказаНаСайте'].nunique(),
        'Выручка': x[(x.НовыйСтатус=='Доставлен')&(x.Статус=='Доставлен')&(x.Отменено=='Нет')&(x.КоличествоПроданоКлиенту>0)]['СуммаСтроки'].sum(),
        'КоличествоТоваров': x['Количество'].sum(),
        'СредняяМаржа': x[(x.НовыйСтатус=='Доставлен')&(x.Статус=='Доставлен')&(x.Отменено=='Нет')&(x.КоличествоПроданоКлиенту>0)]['Маржа'].mean(),
        'ДоляВыкупленных': x[(x.НовыйСтатус=='Доставлен')&(x.Статус=='Доставлен')&(x.Отменено=='Нет')&(x.КоличествоПроданоКлиенту>0)]['КоличествоПроданоКлиенту'].sum() / x['Количество'].sum(),
        'ПроцентОтмен': x[(~(x.НовыйСтатус=='Доставлен'))|(~(x.Статус=='Доставлен'))|(~(x.Отменено=='Нет'))|(~(x.КоличествоПроданоКлиенту>0))]['Отменено'].count() / x['Отменено'].count(),
        'КоличествоЧековВсе': x['НомерЗаказаНаСайте'].nunique(),
        'СреднийЧекВсе': x['СуммаСтроки'].sum() / x['НомерЗаказаНаСайте'].nunique(),
        'СреднееЧислоТоваровЧекаВсе': x['Количество'].sum() / x['НомерЗаказаНаСайте'].nunique(),
        'ВыручкаВся': x['СуммаСтроки'].sum(),
        'СредняяМаржаВся': x['Маржа'].mean(),
    }
    out = pd.Series(result).round(3)
    return out.fillna(0)

In [192]:
df_type = pd.crosstab(df_all['Телефон_new'],
            df_all['Тип'],
            normalize='index').round(3)

In [193]:
df_group2 = pd.crosstab(df_all['Телефон_new'],
            df_all['Группа2'],
            normalize='index').round(3)

In [194]:
df1 = pd.merge(df_type, df_group2, on ='Телефон_new')

In [195]:
df_geo = pd.crosstab(df_all['Телефон_new'],
            df_all['Гео'],
            normalize='index').round(3)

In [196]:
df2 = pd.merge(df1, df_geo, on ='Телефон_new')

In [197]:
df_region = pd.crosstab(df_all['Телефон_new'],
            df_all['Регион'],
            normalize='index').round(3)

In [198]:
df3 = pd.merge(df2, df_region, on ='Телефон_new')

In [199]:
df_sum = df_all.groupby('Телефон_new').apply(summary)

In [200]:
df_finall = pd.merge(df_sum, df3, on ='Телефон_new')

In [201]:
df_finall.replace(np.inf, 0, inplace=True)

In [202]:
df_finall

,КоличествоЧеков,СреднийЧек,СреднееЧислоТоваровЧека,Выручка,КоличествоТоваров,СредняяМаржа,ДоляВыкупленных,ПроцентОтмен,КоличествоЧековВсе,СреднийЧекВсе,...,МО,Москва,Регионы,CENTRAL,FAR EAST,NORTH,PRIVOLZIE,SIBERIA,SOUTHERN,URAL
Телефон_new,,,,,,,,,,,,,,,,,,,,,
0,517.0,2940.555,3.634,1520267.01,7150.0,214.408,0.263,0.756,2256.0,3376.090,...,0.142,0.451,0.407,0.673,0.002,0.093,0.101,0.026,0.067,0.039
32555749-545749525150 .,1.0,1039.000,5.000,1039.00,5.0,34.932,1.000,0.000,1.0,1039.000,...,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000
55494851-56495152535476,1.0,1630.000,1.000,1630.00,1.0,376.020,1.000,0.000,1.0,1630.000,...,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000
55495051-52535455565770,0.0,0.000,0.000,0.00,18.0,0.000,0.000,1.000,18.0,8328.444,...,0.000,0.000,1.000,0.889,0.000,0.000,0.000,0.000,0.111,0.000
55505353-48575248535072,0.0,0.000,0.000,0.00,1.0,0.000,0.000,1.000,1.0,7666.000,...,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57505449-54565157515496,0.0,0.000,0.000,0.00,2.0,0.000,0.000,1.000,1.0,1122.000,...,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000
57545653-57554953575092,0.0,0.000,0.000,0.00,26.0,0.000,0.000,1.000,1.0,8684.000,...,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000
57545657-49575257555799,0.0,0.000,0.000,0.00,3.0,0.000,0.000,1.000,1.0,3704.000,...,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000


In [204]:
df_finall.to_excel("витрина.xlsx") 